In [1]:
# Testing Cell
import aviary.api as av
from aviary.api import Settings
from aviary.utils.doctape import glue_variable, check_value

HEIGHT_ENERGY = av.EquationsOfMotion.HEIGHT_ENERGY
glue_variable('height_energy', HEIGHT_ENERGY.name, md_code=True)

PAYLOAD_RANGE = av.Settings.PAYLOAD_RANGE

str_alternate_snippet = f'```\n{PAYLOAD_RANGE}, {True}\n```'
glue_variable('payload_range_toggle', str_alternate_snippet, md_code=True)


glue_variable('operating_mass', av.Aircraft.Design.OPERATING_MASS, md_code=True)
glue_variable('total_payload_mass', av.Aircraft.CrewPayload.TOTAL_PAYLOAD_MASS, md_code=True)
glue_variable('design_gross_mass', av.Mission.Design.GROSS_MASS, md_code=True)
glue_variable('fuel_capacity', av.Aircraft.Fuel.TOTAL_CAPACITY, md_code=True)

file_path3 = av.get_path('models/aircraft/test_aircraft/aircraft_for_bench_FwFm.csv')

glue_variable('objectives_fuel', av.Mission.Objectives.FUEL, md_code=True)
glue_variable('objectives_range', av.Mission.Objectives.RANGE, md_code=True)
glue_variable('summary_gross_mass', av.Mission.Summary.GROSS_MASS, md_code=True)

`HEIGHT_ENERGY`

````
settings:payload_range, True
````

`aircraft:design:operating_mass`

`aircraft:crew_and_payload:total_payload_mass`

`mission:design:gross_mass`

`aircraft:fuel:total_capacity`

`mission:objectives:fuel`

`mission:objectives:range`

`mission:summary:gross_mass`

# Payload Range Functionality. 

## Overview

Payload Range functionality is currently available for the {glue:md}`height_energy` missions and FLOPS mass systems.

A payload-range diagram is an aircraft performance visualization that illustrates the trade-off between how much weight the aircraft can carry in terms of passengers and cargo (payload) vs how far it can fly (range).
The payload-range shows 3 different points:
- **Maximum Payload Point**: The maximum structural payload the aircraft can carry with sufficient fuel to achieve its design range, this is the 'sizing' mission.
- **Maximum Fuel Point**: The maximum fuel the aircraft can hold with additional payload up to the aircraft's max takeoff weight. 
- **Ferry Range Point**: The maximum fuel the aircraft can carry with zero payload to achieve maximum possible range. 

After a successfull sizing mission (**Maximum Payload Point**) the payload-range function calculates the required aircraft loading conditions and runs an off-design fallout mission for reach of the **Maximum Fuel Point** and **Ferry Range Points**. 
The fallout missions take payload, `phase_info` and `mission_mass` of the aircraft as inputs and solve for the maximum achievable range. 

It is assumed that the `phase_info` contains a standard climb, cruise, descent phases and Aviary will automatically adjust the `time_duration_bound` of the cruise phase to elongate the trajectory for the longer off-design missions.

```{note}
We highly recommend that users first read through [Off Design Missions](off_design_missions) to understand the underlying mechanics of how Aviary runs off-design missions.
```


Next, the system automatically recalculates the {glue:md}`total_payload_mass` for the off-design missions:

- **Maximum Fuel Point**: The payload is assumed to be proportionally reduced in all categories to accommodate for the additional fuel up to capacity. 
The `mission_mass` is set to the {glue:md}`design_gross_mass` of the sizing mission/Maximum Payload Point.

- **Ferry Range Point**: The payload is reduced to one passenger (due to off-design limitations) with the `mission_mass` argument set to the {glue:md}`operating_mass` plus {glue:md}`fuel_capacity`.

After which the **Maximum Fuel Point** and **Ferry Range Point** are run immediately after the **Maximum Payload Point**.

```{note}
Aviary is structured to give a payload-range diagram even a **Maximum Fuel Point** is not feasable, in those cases just the **Maximum Payload Point** and **Ferry Range Point** are plotted. 
```

## Running a payload/range analysis 

### Input CSV File
There are currently two methods to run a payload-range analysis. 
The first method is to add the following line to the Aviary inputs CSV file. 

```{glue:md} payload_range_toggle
:format: myst
```


```{note}
This will result in Aviary running a payload-range analysis every time that specific aircraft is sized. 
```

### Manual Function Call
The second method is to manually call the `run_payload_range()` function after running a sizing mission. 
If verbosity is set to 2 or higher, the function will print the payload-range values directly into the terminal.

`(prob_fallout_max_fuel_plus_payload, prob_fallout_ferry) = prob.run_payload_range(verbosity=2)`

- Verbosity: optional parameter, default value = 1. 
    Prints payload-range to terminal if Verbosity ≥ 2.

The function returns a tuple containing two Aviary problem objects. 
- `prob_fallout_max_fuel_plus_payload`: off-design mission ran for the **Maximum Fuel Point**
- `prob_fallout_ferry`: off-design mission configured for **Ferry Range Point**


### Output and visualization. 
The payload-range values are saved within the `payload_range_data.csv` file within the reports folder of the sizing mission's Aviary output directory. 
The information is also visualized within the `Payload/Range Diagram` tab in the Aviary dashboard for the sizing mission.  

```{note}
Aviary will only save and illustrate a payload/range analysis if the sizing and two off-design missions converged to a valid solution.
```

### Example:

The user can then within the script used to run the sizing mission interrogate the off-design missions. 
An example of a payload/range being manually executed is available below. 

In [2]:
%%capture

import aviary.api as av


# defaults for height energy based phases

phase_info = av.default_height_energy_phase_info

##################
# Sizing Mission #
##################
prob = av.AviaryProblem()

# Load aircraft and options data from user
# Allow for user overrides here
prob.load_inputs('models/test_aircraft/aircraft_for_bench_FwFm.csv', phase_info)

# You can either set the payload/range toggle 'settings:payload_range' to True in the input file,
# or you can manually run the payload/range at the end of the script.

# Preprocess inputs
prob.check_and_preprocess_inputs()
prob.add_pre_mission_systems()
prob.add_phases()
prob.add_post_mission_systems()

# Link phases and variables
prob.link_phases()
prob.add_driver('SLSQP', max_iter=50)
prob.add_design_variables()

# Load optimization problem formulation
# Detail which variables the optimizer can control
prob.add_objective()
prob.setup()
prob.set_initial_guesses()
prob.run_aviary_problem()

# Manually run the payload/range analysis
# The script will only print values to the terminal if verbosity is set to VERBOSE (2) or higher
# The analysis can return the max_fuel_plus_payload & ferry fallout missions for further analysis
(prob_fallout_max_fuel_plus_payload, prob_fallout_ferry) = prob.run_payload_range(verbosity=2)


# This returns the two Aviary problem objects that can then be interrogated like any regular Aviary object.
max_payload_plus_fuel__payload = prob.get_val(av.Aircraft.CrewPayload.TOTAL_PAYLOAD_MASS, 'lbm')[0]
max_fuel_plus_payload__payload = prob_fallout_max_fuel_plus_payload.get_val(
    av.Aircraft.CrewPayload.TOTAL_PAYLOAD_MASS, 'lbm'
)[0]
ferry__payload = prob_fallout_ferry.get_val(av.Aircraft.CrewPayload.TOTAL_PAYLOAD_MASS, 'lbm')[0]


max_payload_plus_fuel__range = prob.get_val(av.Mission.Summary.RANGE, 'nmi')[0]
max_fuel_plus_payload__range = prob_fallout_max_fuel_plus_payload.get_val(
    av.Mission.Summary.RANGE, 'nmi'
)[0]
ferry__range = prob_fallout_ferry.get_val(av.Mission.Summary.RANGE, 'nmi')[0]

# In Aviary's terminal output, the manual and automatic prints will be exactly the same.
print('--------------------')
print('Manual Payload/Range')
print('--------------------')

print('Range', 0, max_payload_plus_fuel__range, max_fuel_plus_payload__range, ferry__range)
print(
    'Payload',
    max_payload_plus_fuel__payload,
    max_payload_plus_fuel__payload,
    max_fuel_plus_payload__payload,
    ferry__payload,
)

In [3]:
# Testing Cell
import os
import aviary.api as av
from aviary.utils.doctape import check_contains

# make sure off_design_example.py exists in aviary/examples folder
off_design_examples = av.get_path(os.path.join('examples'))
check_contains(
    ('run_off_design_example.py'),
    os.listdir(off_design_examples),
    error_string='{var} not in ' + str(off_design_examples),
    error_type=FileNotFoundError,
)

# make sure run_level2_example.py exists in aviary/examples folder
run_level2_examples = av.get_path(os.path.join('examples'))
check_contains(
    ('run_level2_example.py'),
    os.listdir(run_level2_examples),
    error_string='{var} not in ' + str(run_level2_examples),
    error_type=FileNotFoundError,
)